In [ ]:
# Notebook: pdf_extraction.ipynb

import os
import sys
from typing import Dict, List, Optional
import logging
import pathlib

# Configuración de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Intentamos usar diferentes bibliotecas de extracción de PDFs
try:
    from pypdf import PdfReader
    USE_PYPDF = True
except ImportError:
    USE_PYPDF = False
    logger.warning("pypdf no está instalado. Usando alternativa.")

try:
    from pdfminer.high_level import extract_text as pdfminer_extract_text
    USE_PDFMINER = True
except ImportError:
    USE_PDFMINER = False
    logger.warning("pdfminer.six no está instalado. Usando alternativa.")


def extract_text_with_pypdf(pdf_path: str) -> str:
    """Extrae texto de un PDF usando pypdf."""
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text


def extract_text_with_pdfminer(pdf_path: str) -> str:
    """Extrae texto de un PDF usando pdfminer."""
    return pdfminer_extract_text(pdf_path)


def extract_text_from_pdf(pdf_path: str) -> str:
    """
    Extrae texto de un archivo PDF utilizando la mejor biblioteca disponible.
    """
    logger.info(f"Extrayendo texto de: {os.path.basename(pdf_path)}")
    
    if USE_PYPDF:
        try:
            text = extract_text_with_pypdf(pdf_path)
            if text.strip():
                return text
            logger.warning("PyPDF no extrajo texto. Probando con alternativa.")
        except Exception as e:
            logger.error(f"Error con PyPDF: {e}")
    
    if USE_PDFMINER:
        try:
            return pdfminer_extract_text(pdf_path)
        except Exception as e:
            logger.error(f"Error con PDFMiner: {e}")
    
    # Si llegamos aquí, ninguna biblioteca funcionó
    logger.error(f"No se pudo extraer texto de {pdf_path}")
    return ""


def process_pdf_directory(pdf_dir: str, output_dir: Optional[str] = None) -> Dict[str, str]:
    """
    Procesa todos los PDFs en un directorio y extrae su texto.
    """
    if not os.path.exists(pdf_dir):
        raise FileNotFoundError(f"El directorio {pdf_dir} no existe")
    
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    pdf_files = [f for f in os.listdir(pdf_dir) if f.lower().endswith('.pdf')]
    logger.info(f"Se encontraron {len(pdf_files)} archivos PDF en {pdf_dir}")
    
    results = {}
    
    for pdf_file in pdf_files:
        pdf_path = os.path.join(pdf_dir, pdf_file)
        text = extract_text_from_pdf(pdf_path)
        
        # Guardar en resultados
        results[pdf_file] = text
        
        # Guardar en archivo si se especificó un directorio de salida
        if output_dir and text:
            text_filename = os.path.splitext(pdf_file)[0] + '.txt'
            text_path = os.path.join(output_dir, text_filename)
            
            with open(text_path, 'w', encoding='utf-8') as f:
                f.write(text)
            
            logger.info(f"Texto guardado en: {text_path}")
    
    return results

# Obtener el directorio raíz del proyecto (ajustar según dónde esté tu notebook)
# Si el notebook está en src/notebooks/extraction, entonces:
current_dir = pathlib.Path(os.getcwd())
# Suponiendo que estás ejecutando desde un notebook en la raíz del proyecto:
project_root = current_dir.parent

# Definir rutas absolutas
pdf_dir = os.path.join(current_dir.parent, "bibliography")
output_dir = os.path.join(current_dir, "chunks", "raw_text")

# Imprimir las rutas para verificar
print(f"Directorio de PDFs: {pdf_dir}")
print(f"Directorio de salida: {output_dir}")

# Asegúrate de que el directorio de salida exista
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Preguntar al usuario si las rutas son correctas antes de continuar
confirmation = input("¿Son correctas estas rutas? (s/n): ")
if confirmation.lower() != 's':
    print("Proceso cancelado.")
else:
    # Ejecutar la extracción
    results = process_pdf_directory(pdf_dir, output_dir)
    print(f"Procesados {len(results)} archivos PDF")

    # Mostrar ejemplo del primer PDF procesado (primeras 500 palabras)
    if results:
        first_pdf = list(results.keys())[0]
        first_text = results[first_pdf]
        print(f"\nEjemplo de texto extraído de {first_pdf}:")
        print(first_text[:1000] + "...")